# Test management of organization members

In [ ]:
import sqlmodel as sqm

# Test assets
from laminhub_rest.core.account import create_organization_account
from laminhub_rest.routers.account import get_account_organizations

# Test utils
from laminhub_rest.utils._test import create_test_auth, create_test_account
from laminhub_rest.connector import connect_hub_with_auth
from laminhub_rest.main import client
from laminhub_rest._clean_ci import clean_ci
from laminhub_rest.utils._id import base62

# Parametrize

In [ ]:
# Create account 1
auth_1 = create_test_auth()
access_token_1 = auth_1["access_token"]
account_1 = create_test_account(handle=auth_1["handle"], access_token=access_token_1)

# Create account 2
auth_2 = create_test_auth()
access_token_2 = auth_2["access_token"]
account_2 = create_test_account(handle=auth_2["handle"], access_token=access_token_2)

# Create organization account
org_handle_1 = f"lamin.ci.org.{base62(8)}"
organization = create_organization_account(
    handle=org_handle_1, _access_token=access_token_1
)
hub = connect_hub_with_auth(access_token=access_token_1)
organization_id = (
    hub.table("account").select("id").eq("handle", org_handle_1).execute().data[0]["id"]
)

## Test fetching of account's organizations

In [ ]:
organizations = client.get(
    f"/account/resources/organizations/{account_1['handle']}",
    headers={"authentication": f"Bearer {access_token_1}"},
).json()

In [ ]:
assert len(organizations) == 1
assert organizations[0]["account"]["handle"] == org_handle_1

## Test addition of a member 

In [ ]:
response = client.post(
    f"/organization/resources/members/{organization_id}/?user_id={account_2['id']}",
    headers={"authentication": f"Bearer {access_token_1}"},
).json()

assert response == "success"

## Test update of a member 

In [ ]:
response = client.put(
    f"organization/resources/members/{organization_id}/?user_id={account_2['id']}&role=owner",
    headers={"authentication": f"Bearer {access_token_1}"},
).json()

assert response == "success"

## Test removal of a member

In [ ]:
response = client.delete(
    f"organization/resources/members/{organization_id}/?user_id={account_2['id']}",
    headers={"authentication": f"Bearer {access_token_1}"},
).json()

assert response == "success"

## Clean up test assets

In [ ]:
clean_ci()